# Auth Weekly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
import pandas as pd
from datetime import datetime, timedelta
from cloud_sql_connector import DBConfig, getconn
import pg8000
import sqlalchemy

This will create the connection to the database

In [ ]:
# Initialize DBConfig for Cloud SQL connection
config = DBConfig(
    instance_name=os.getenv('DB_INSTANCE_CONNECTION_NAME'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    ip_type="public",
    schema="auth"
)

def get_conn():
    """Create a connection to Google Cloud SQL using the custom cloud-sql-connector."""
    return getconn(config)

# Create SQLAlchemy engine using Cloud SQL connector
engine = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=get_conn,
)

print("Cloud SQL engine created successfully!")

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(sqlalchemy.text("SELECT 1"))
        print("✅ DB connection OK:", result.fetchone())
except Exception as e:
    print("❌ Connection failed:", e)

In [ ]:
#%% File setup
datestr = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
filename = 'auth_weekly_stats_till_' + datestr + '.csv'

In [ ]:
#%% Weekly Accounts
query = """
SELECT count(*) FILTER (WHERE type_code = 'BASIC') AS BASIC,
       count(*) FILTER (WHERE type_code = 'PREMIUM') AS PREMIUM
FROM orgs 
WHERE status_code='ACTIVE'
AND date(created at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(created at time zone 'utc' at time zone 'pst') <= date(current_date - 1)
"""
stat_accounts_weekly_completed = pd.read_sql(query, engine)

with open(filename, 'w') as f:
    stat_accounts_weekly_completed.insert(0, "Weekly Number of New Accounts:", [''], True)
    stat_accounts_weekly_completed.to_csv(f, sep=',', encoding='utf-8', index=False)


In [ ]:
#%% Total Accounts
query = """
SELECT count(*) FILTER (WHERE type_code = 'BASIC') AS BASIC,
       count(*) FILTER (WHERE type_code = 'PREMIUM') AS PREMIUM
FROM orgs 
WHERE status_code='ACTIVE'
"""
stat_accounts_total_completed = pd.read_sql(query, engine)

with open(filename, 'a') as f:      
    f.write('\n\n')
    stat_accounts_total_completed.insert(0, "Total Number of Accounts:", [''], True)
    stat_accounts_total_completed.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
#%% Weekly Login Options
query = """
SELECT COUNT(*) FILTER (WHERE a.login_source = 'BCEID') AS BCEID,
       COUNT(*) FILTER (WHERE a.login_source = 'BCSC') AS BCSC
FROM orgs o
JOIN account_login_options a ON o.id=a.org_id
WHERE o.status_code='ACTIVE' 
AND a.is_active='TRUE'
AND date(o.created at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(o.created at time zone 'utc' at time zone 'pst') <= date(current_date - 1)
"""
stat_accounts_login_option_weekly_completed = pd.read_sql(query, engine)

df = stat_accounts_login_option_weekly_completed
df['bceid_%'] = round((df.BCEID / (df.BCEID + df.BCSC)) * 100, 2)
df['bcsc_%'] = round((df.BCSC / (df.BCEID + df.BCSC)) * 100, 2)

with open(filename, 'a') as f:
    f.write('\n\n')
    df.insert(
        0,
        "New Accounts for BCeID and BCSC from " + 
        datetime.strftime(datetime.now() - timedelta(8), '%Y-%m-%d') + " to " + 
        datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d') + ":",
        [''],
        True
    )
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
#%% Weekly Login Options Last Year
query = """
SELECT COUNT(*) FILTER (WHERE a.login_source = 'BCEID') AS BCEID,
       COUNT(*) FILTER (WHERE a.login_source = 'BCSC') AS BCSC
FROM orgs o
JOIN account_login_options a ON o.id=a.org_id
WHERE o.status_code='ACTIVE' 
AND a.is_active='TRUE'
AND date(o.created at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 years' - interval '1 weeks')
AND date(o.created at time zone 'utc' at time zone 'pst') <= date(current_date - 1 - interval '1 years')
"""
stat_accounts_login_option_weekly_last_year_completed = pd.read_sql(query, engine)

df = stat_accounts_login_option_weekly_last_year_completed
df['bceid_%'] = round((df.BCEID / (df.BCEID + df.BCSC)) * 100, 2)
df['bcsc_%'] = round((df.BCSC / (df.BCEID + df.BCSC)) * 100, 2)

with open(filename, 'a') as f:
    f.write('\n\n')
    df.insert(
        0,
        "New Accounts for BCeID and BCSC last year from " + 
        datetime.strftime(datetime.now() - timedelta(373), '%Y-%m-%d') + " to " + 
        datetime.strftime(datetime.now() - timedelta(366), '%Y-%m-%d') + ":",
        [''],
        True
    )
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
#%% Past 20 Years Login Options
query = """
SELECT SUBSTRING(CAST(date_part('year', o.created) AS VARCHAR), 1, 4) AS YEAR,
       COUNT(*) FILTER (WHERE a.login_source = 'BCEID') AS BCEID,
       COUNT(*) FILTER (WHERE a.login_source = 'BCSC') AS BCSC
FROM orgs o
JOIN account_login_options a ON o.id=a.org_id
WHERE o.status_code='ACTIVE' 
AND a.is_active='TRUE'
AND date(o.created at time zone 'utc' at time zone 'pst') > date(current_date - interval '20 years')
GROUP BY date_part('year', o.created)
ORDER BY date_part('year', o.created) DESC;
"""
stat_accounts_login_option_pass_years_completed = pd.read_sql(query, engine)

df = stat_accounts_login_option_pass_years_completed
df['bceid_%'] = round((df.BCEID / (df.BCEID + df.BCSC)) * 100, 2)
df['bcsc_%'] = round((df.BCSC / (df.BCEID + df.BCSC)) * 100, 2)

arr = ['' for _ in range(len(df))]
with open(filename, 'a') as f:      
    f.write('\n\n')
    df.insert(0, "Total Number of BCeID and BCSC Accounts by Year:", arr , True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
#%% Weekly Login Users
query = """
SELECT count(*) FILTER (WHERE login_source = 'BCEID') AS BCEID,
       count(*) FILTER (WHERE login_source = 'BCSC') AS BCSC
FROM users
WHERE date(login_time at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(login_time at time zone 'utc' at time zone 'pst') <= date(current_date - 1)
"""
stat_users_weekly_completed = pd.read_sql(query, engine)

with open(filename, 'a') as f:      
    f.write('\n\n')
    stat_users_weekly_completed.insert(0, "Weekly Number of Login Users:", [''], True)
    stat_users_weekly_completed.to_csv(f, sep=',', encoding='utf-8', index=False)


In [ ]:
#%% Total Login Users
query = """
SELECT count(*) FILTER (WHERE login_source = 'BCEID') AS BCEID,
       count(*) FILTER (WHERE login_source = 'BCSC') AS BCSC
FROM users
"""
stat_users_total_completed = pd.read_sql(query, engine)

with open(filename, 'a') as f:      
    f.write('\n\n')
    stat_users_total_completed.insert(0, "Total Number of Login Users:", [''], True)
    stat_users_total_completed.to_csv(f, sep=',', encoding='utf-8', index=False)